# Import Dependencies

In [1]:
from sqlalchemy import create_engine
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Connect to sqlite database
engine = create_engine("sqlite:////filepath/soccer.sqlite", echo=False)
conn = engine.connect()

In [3]:
# Confirm connection by printing tables
engine.table_names()

['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes',
 'sqlite_sequence']

# Examine Data

In [4]:
sql_view = "SELECT * FROM Match LIMIT 3;"

In [5]:
# Run query
view_data = pd.read_sql(sql_view, conn)
# Displaying data
view_data

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.73,3.4,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.8,3.3,3.75,None,None,None,1.70,3.30,4.33,1.90,3.3,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.95,3.2,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.9,3.2,3.50,None,None,None,1.83,3.30,3.60,1.95,3.3,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.38,3.3,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.5,3.2,2.50,None,None,None,2.50,3.25,2.40,2.63,3.3,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75


# Example 1: Simple Subsquery in <font color="red">SELECT</font> Clause

<strong>Generic Synthax</strong><br>
SELECT column(s)<br>
FROM subquery<br>
WHERE conditions;<br>

## <font color="blue">Goal: </font><br>Create a table that compares the average goals scores by a league to the overall average goals scored.

## Step 1: <br>Create the Inner Query
### Retrieve the overall average goals scored in the 2012-2013 season

In [6]:
sql_subquery1 = """SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) AS 'overall_average'
                 FROM Match
                 WHERE season = '2012/2013';"""

In [7]:
# Run query
subquery1 = pd.read_sql(sql_subquery1, conn)
# Displaying data
subquery1

,overall_average
0,2.77


<font color="blue">Note:</font><br>
This query returned a calculated aggregated value.<br>
It also only returned a single value.

## Step 2: Place the Subquery in the Main Query <code>SELECT</code> Clause.

 - Make sure to use the same filter conditions in the main query that was used in the subquery.

In [8]:
sql_outter1 = """
                SELECT l.name AS league,
                ROUND(AVG(m.home_team_goal + m.away_team_goal), 2) AS avg_goals,
                (SELECT ROUND(AVG(home_team_goal + away_team_goal), 2)
                    FROM Match
                    WHERE season = '2012/2013') AS overall_avg
                FROM League as l
                LEFT JOIN Match as m
                ON l.country_id = m.country_id
                WHERE season = '2012/2013'
                GROUP BY league
                ORDER BY avg_goals DESC
                LIMIT 5;
                """

## Step 3: Display Query Result

In [9]:
# Run query
view_data1 = pd.read_sql(sql_outter1, conn)
# Displaying data
view_data1

,league,avg_goals,overall_avg
0,Netherlands Eredivisie,3.15,2.77
1,Belgium Jupiler League,2.93,2.77
2,Germany 1. Bundesliga,2.93,2.77
3,Spain LIGA BBVA,2.87,2.77
4,England Premier League,2.80,2.77


## <font color="blue">The Outcome</font> The five leagues with average goals > than the average overall goals in 2012/2013 season were: 
<ol>
    <li>Netherlands</li>
    <li>Belgium Jupiler League</li>
    <li>Germany 1. Bundesliga	</li>
    <li>Spain LIGA BBVA</li>
    <li>England Premier league</li>    
</ol>   

### <font color="blue">Explanation:</font><br>

<strong>The INNER Subquery</strong><br>
<code>SELECT ROUND(AVG(home_team_goal + away_team_goal), 2) AS 'overall_average'
FROM Match
WHERE season = '2012/2013'</code>
<br>
This subquery selects the columns <br>
 1. from Match table: home_team_goal and away_team_goal are selected
 2. The WHERE clause returns the rows where seasons = '2012/2013' 
 2. It them calculates the average by adding these two columns
 3. The returned value is rounded to 2 decimal places
 4. The output is labeled 'overall_average'
<br>
<br>
<strong>The MAIN query</strong><br>
<code>SELECT l.name AS league,
ROUND(AVG(m.home_team_goal + m.away_team_goal), 2) AS avg_goals,
overall_avg #this was redacted for simplicity (inner query code)
FROM League as l
LEFT JOIN Match as m
ON l.country_id = m.country_id
WHERE season = '2012/2013'
GROUP BY league
ORDER BY avg_goals DESC
LIMIT 5;</code>
<br>
<br>

<strong>SELECT Statement</strong>
The MAIN query selected the following columns<br>
 1. <code>l.name AS league</code><br>
 2. <code>overall_avg</code> from inner subquery
 <br>
<strong>Tables and Join Used</strong>
 1. <code>League as l</code><br> Main table is League and is aliased as 'l'
 2. <code>LEFT JOIN Match as m</code> This code merges data from tge Match table to League table
 3. <code>ON l.country_id = m.country_id</code>This specifies which columns will be used for the merger.
 <br>
<strong>Filtering Steps with WHERE clause</strong>
The main query then filteres by the <code>WHERE</code> clause, and returns all instances where the <code>season</code> contains '2012/2013'.
<strong>Data Organization and return limits</strong>
 1. <code>GROUP BY league</code><br> This will aggregate rows by the league value
 2. <code>ORDER BY avg_goals DESC</code> This will sort the data by avg_goals column in descending value
 3. <code>LIMIT 5</code> This will dispaly only 5 instances.

## What to remember

 - Make sure the filter conditions are consistent in the main and in the inner query.
 - Subqueries in SELECT clause return a single aggregated value

# Example 2: Conducting Calculations in <font color="red">SELECT</font> Subqueries

<strong>Generic Synthax</strong><br>
SELECT column(s), calcualation involving subquery<br>
FROM table(s);<br>

## <font color="blue">Goal: </font><br>Create a table that adds a column that directly compares the average goals scored by a league to the total overall average goal, by subtracting the overall average from the subquery.

## Step 1: <br>Create the Inner Query
### Retrieve the overall average goals scored in the 2012-2013 season

In [10]:
sql_subquery2 = """SELECT AVG(home_team_goal + away_team_goal) AS overall_average
                    FROM Match 
                    WHERE season = '2012/2013';"""

In [11]:
# Run query
subquery2 = pd.read_sql(sql_subquery2, conn)
# Displaying data
subquery2

,overall_average
0,2.772699


## Step 2: Perform calculation on the <code>SELECT</code> Clause Subquery in the Main Query .

 - Make sure to use the same filter conditions in the main query that was used in the subquery.

In [12]:
sql_outter2 = """
                SELECT l.name AS league,
                ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals,
                ROUND(AVG(m.home_team_goal + m.away_team_goal) - 
                (SELECT AVG(home_team_goal + away_team_goal)
                FROM Match 
                WHERE season = '2012/2013'),2) AS diff
                FROM League AS l
                LEFT JOIN Match AS m
                ON l.country_id = m.country_id
                -- Only include 2012/2013 results
                WHERE season = '2012/2013'
                GROUP BY l.name;"""

## Step 3: Display Query Result

In [13]:
# Run query
view_data1 = pd.read_sql(sql_outter2, conn)
# Displaying data
view_data1

,league,avg_goals,diff
0,Belgium Jupiler League,2.93,0.16
1,England Premier League,2.80,0.02
2,France Ligue 1,2.54,-0.23
3,Germany 1. Bundesliga,2.93,0.16
4,Italy Serie A,2.64,-0.13
5,Netherlands Eredivisie,3.15,0.38
6,Poland Ekstraklasa,2.49,-0.28
7,Portugal Liga ZON Sagres,2.78,0.01
8,Scotland Premier League,2.73,-0.04
9,Spain LIGA BBVA,2.87,0.10


### <font color="blue">Explanation:</font><br>

<strong>The INNER Subquery</strong><br>
<code>SELECT AVG(home_team_goal + away_team_goal) AS overall_average
FROM Match 
WHERE season = '2012/2013'</code>
<br>
This subquery selects the columns home_team_goal and away_team_goal <br>
 1. from Match table: home_team_goal and away_team_goal are selected
 2. The WHERE clause returns the rows where seasons = '2012/2013' 
 3. It them calculates the average by adding these two columns
 4. The output is labeled 'overall_average'
<br>
<br>
<strong>The MAIN query</strong><br>
<code>SELECT l.name AS league,
ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals,
ROUND(AVG(m.home_team_goal + m.away_team_goal) - 
(SELECT AVG(home_team_goal + away_team_goal)
FROM Match 
WHERE season = '2012/2013'),2) AS diff
FROM League AS l
LEFT JOIN Match AS m
ON l.country_id = m.country_id
-- Only include 2012/2013 results
WHERE season = '2012/2013'
GROUP BY league;</code>
<br>
<br>
<strong>SELECT Statement</strong>
The MAIN query selected the following columns<br>
 1. <code>l.name AS league</code><br>
 2. <code>ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals</code> calculated field from the Match table
 3. <strong>Step that performs calculation on subquery, and creates a new column: </strong><br>
Simplified:
<code>ROUND(AVG(m.home_team_goal + m.away_team_goal) - subquery AS diff</code><br>
Full length:
<code>ROUND(AVG(m.home_team_goal + m.away_team_goal) - 
(SELECT AVG(home_team_goal + away_team_goal)
FROM Match 
WHERE season = '2012/2013'),2) AS diff</code><br>
 <br>
<strong>Tables and Join Used</strong>
 1. <code>League as l</code><br> Main table is League and is aliased as 'l'
 2. <code>LEFT JOIN Match as m</code> This code merges data from the Match table to League table
 3. <code>ON l.country_id = m.country_id</code>This specifies which columns will be used for the merger.
 <br>
<strong>Filtering Steps with WHERE clause</strong>
The main query then filteres by the <code>WHERE</code> clause, and returns all instances where the <code>season</code> contains '2012/2013'.
<strong>Data Organization and return limits</strong>
 1. <code>GROUP BY league</code><br> This will aggregate rows by the league value